In [2]:
import pandas as pd

In [4]:
team='Bayern Munich'

In [3]:
df=pd.read_csv('C:/Users/pedro/Projetos/bundesbet/src/data/game_stats.csv')

In [4]:
df.columns

Index(['url', 'round', 'home_team', 'away_team', 'goals_home_final',
       'goals_away_final', 'final_result', 'goals_home_1half',
       'goals_away_1half', '1half_result', 'ball_possessionhome',
       'ball_possessionaway', 'goal_attemptshome', 'goal_attemptsaway',
       'shots_on_goalhome', 'shots_on_goalaway', 'shots_off_goalhome',
       'shots_off_goalaway', 'blocked_shotshome', 'blocked_shotsaway',
       'free_kickshome', 'free_kicksaway', 'corner_kickshome',
       'corner_kicksaway', 'offsideshome', 'offsidesaway',
       'goalkeeper_saveshome', 'goalkeeper_savesaway', 'foulshome',
       'foulsaway', 'total_passeshome', 'total_passesaway', 'tackleshome',
       'tacklesaway', 'attackshome', 'attacksaway', 'dangerous_attackshome',
       'dangerous_attacksaway', 'yellow_cardshome', 'yellow_cardsaway',
       'red_cardshome', 'red_cardsaway'],
      dtype='object')